# 第2章 感知机

1．感知机是根据输入实例的特征向量$x$对其进行二类分类的线性分类模型：

$$
f(x)=\operatorname{sign}(w \cdot x+b)
$$

感知机模型对应于输入空间（特征空间）中的分离超平面$w \cdot x+b=0$。

2．感知机学习的策略是极小化损失函数：

$$
\min _{w, b} L(w, b)=-\sum_{x_{i} \in M} y_{i}\left(w \cdot x_{i}+b\right)
$$

损失函数对应于误分类点到分离超平面的总距离。

3．感知机学习算法是基于随机梯度下降法的对损失函数的最优化算法，有原始形式和对偶形式。算法简单且易于实现。原始形式中，首先任意选取一个超平面，然后用梯度下降法不断极小化目标函数。在这个过程中一次随机选取一个误分类点使其梯度下降。
 
4．当训练数据集线性可分时，感知机学习算法是收敛的。感知机算法在训练数据集上的误分类次数$k$满足不等式：

$$
k \leqslant\left(\frac{R}{\gamma}\right)^{2}
$$

当训练数据集线性可分时，感知机学习算法存在无穷多个解，其解由于不同的初值或不同的迭代顺序而可能有所不同。


### 二分类模型
$f(x) = sign(w\cdot x + b)$

$\operatorname{sign}(x)=\left\{\begin{array}{ll}{+1,} & {x \geqslant 0} \\ {-1,} & {x<0}\end{array}\right.$

给定训练集：

$T=\left\{\left(x_{1}, y_{1}\right),\left(x_{2}, y_{2}\right), \cdots,\left(x_{N}, y_{N}\right)\right\}$

定义感知机的损失函数 

$L(w, b)=-\sum_{x_{i} \in M} y_{i}\left(w \cdot x_{i}+b\right)$

---
#### 算法

随即梯度下降法 Stochastic Gradient Descent

随机抽取一个误分类点使其梯度下降。

$w = w + \eta y_{i}x_{i}$

$b = b + \eta y_{i}$

当实例点被误分类，即位于分离超平面的错误侧，则调整$w$, $b$的值，使分离超平面向该无分类点的一侧移动，直至误分类点被正确分类

【1】算法实现-原始形式

![title](../images/感知机原始算法.png)

In [1]:
from numpy import *  
import operator
import os



# create a dataset which contains 3 samples with 2 classes  
def createDataSet():  
    # create a matrix: each row as a sample  
    group = array([[3,3], [4,3], [1,1]])  
    labels = [1, 1, -1] # four samples and two classes  
    return group, labels

#classify using perceptron
def perceptronClassify(trainGroup,trainLabels):
    global w, b
    isFind = False  #the flag of find the best w and b
    numSamples = trainGroup.shape[0]
    mLenth = trainGroup.shape[1]
    w = [0]*mLenth
    b = 0
    while(not isFind):
        for i in range(numSamples):
            if cal(trainGroup[i],trainLabels[i]) <= 0:
                print('w:', w, 'b:', b)
                update(trainGroup[i],trainLabels[i])
                break    #end for loop
            elif i == numSamples-1:
                print(w, b)
                isFind = True   #end while loop


def cal(row,trainLabel):
    global w, b
    res = 0
    for i in range(len(row)):
        res += row[i] * w[i]
    res += b
    res *= trainLabel
    return res
def update(row,trainLabel):
    global w, b
    for i in range(len(row)):
        w[i] += trainLabel * row[i]
    b += trainLabel
    
    
if __name__=='__main__':
    g,l = createDataSet()
    perceptronClassify(g,l)


w: [0, 0] b: 0
w: [3, 3] b: 1
w: [2, 2] b: 0
w: [1, 1] b: -1
w: [0, 0] b: -2
w: [3, 3] b: -1
w: [2, 2] b: -2
[1, 1] -3


![title](../images/感知机算法对偶形式.png)

In [2]:
from numpy import *  
import operator
import os



# create a dataset which contains 3 samples with 2 classes  
def createDataSet():  
    # create a matrix: each row as a sample  
    group = array([[3,3], [4,3], [1,1]])  
    labels = [1, 1, -1] # four samples and two classes  
    return group, labels

#classify using DualPerception
def dualPerceptionClassify(trainGroup,trainLabels):
    global a,b
    isFind = False
    numSamples = trainGroup.shape[0]
    #mLenth = trainGroup.shape[1]
    a = [0]*numSamples
    b = 0
    gMatrix = cal_gram(trainGroup)
    while(not isFind):
        for i in range(numSamples):
            if cal(gMatrix,trainLabels,i) <= 0:
                cal_wb(trainGroup,trainLabels)
                update(i,trainLabels[i])
                break
            elif i == numSamples-1:
                cal_wb(trainGroup,trainLabels)
                isFind = True
    

# caculate the Gram matrix
def cal_gram(group):
    mLenth = group.shape[0]
    gMatrix = zeros((mLenth,mLenth))
    for i in range(mLenth):
        for j in range(mLenth):
            gMatrix[i][j] =  dot(group[i],group[j])
    return gMatrix

def update(i,trainLabel):
    global a,b
    a[i] += 1
    b += trainLabel

def cal(gMatrix,trainLabels,key):
    global a,b
    res = 0
    for i in range(len(trainLabels)):
        res += a[i]*trainLabels[i]*gMatrix[key][i]
    res = (res + b)*trainLabels[key]
    return res

#caculator w and b,then print it
def cal_wb(group,labels):
    global a,b
    w=[0]*(group.shape[1])
    h = 0
    for i in range(len(labels)):
        h +=a[i]*labels[i]
        w +=a[i]*labels[i]*group[i]
    print(w,h)
    
    
    
if __name__=='__main__':
    g, l = createDataSet()
    dualPerceptionClassify(g, l)

[0 0] 0
[3 3] 1
[2 2] 0
[1 1] -1
[0 0] -2
[3 3] -1
[2 2] -2
[1 1] -3
